In [1]:
import torch
import torch.nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

import numpy as np
import csv

In [17]:
# Hyper parameters
VALIDATION_SPLIT_RATIO = 0.2
EPOCHS = 2
BATCH_SIZE = 40
LOG_INTERVAL = 10

In [3]:
# Load the data
target = []
images = []
with open("training.csv") as csvfile:
    reader = csv.reader(csvfile)
    first_line = True
    for line in reader:
        if first_line == True:
            first_line = False
            continue
        target.append(np.array([float(x) if x!="" else 0 for x in line[:-1]]))
        images.append(np.array([int(x) for x in line[-1].split(" ")]).reshape(1,96,96))
        
y = np.array(target,dtype="float")
X = np.array(images,dtype="float")

n_images = X.shape[0]

print("Number of images in the entire dataset: %s" % n_images)

Number of images in the entire dataset: 7049


In [4]:
# Split the data for validation
X_train = X[:int((1-VALIDATION_SPLIT_RATIO)*n_images)]/255. - 0.5
y_train = y[:int((1-VALIDATION_SPLIT_RATIO)*n_images)]
X_val = X[int((1-VALIDATION_SPLIT_RATIO)*n_images):]/255. - 0.5
y_val = y[int((1-VALIDATION_SPLIT_RATIO)*n_images):]

n_train = X_train.shape[0]
n_val = X_val.shape[0]

print("Shape of X_train:")
print(X_train.shape)
print("Shape of y_train:")
print(y_train.shape)
print("Number of images in the training dataset: %s" % n_train)
print("Number of images in the validation dataset: %s" % n_val)


Shape of X_train:
(5639, 1, 96, 96)
Shape of y_train:
(5639, 30)
Number of images in the training dataset: 5639
Number of images in the validation dataset: 1410


In [5]:
# Create Dataloaders
import torch.utils.data

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train.astype('float32')).type(torch.FloatTensor),  \
                                               torch.from_numpy(y_train.astype('float32')).type(torch.FloatTensor))
val_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_val.astype('float32')).type(torch.FloatTensor), \
                                             torch.from_numpy(y_val.astype('float32')).type(torch.FloatTensor))

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=BATCH_SIZE)

In [6]:
class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 24, 5)
        self.MPool = torch.nn.MaxPool2d(2, stride=(2,2))
        self.conv2 = torch.nn.Conv2d(24, 36, 5)
        self.conv3 = torch.nn.Conv2d(36, 48, 5)
        self.conv4 = torch.nn.Conv2d(48, 64, 3)
        self.conv5 = torch.nn.Conv2d(64, 64, 3)
        self.conv2_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(64, 500)
        self.fc2 = torch.nn.Linear(500, 90)
        self.fc3 = torch.nn.Linear(90, 30)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.MPool(x)
        x = F.relu(self.conv2(x))
        x = self.MPool(x)
        x = F.relu(self.conv3(x))
        x = self.MPool(x)
        x = F.relu(self.conv4(x))
        x = self.MPool(x)
        x = F.relu(self.conv5(x))
        x = x.view(x.size(0), -1)
        x = self.conv2_drop(F.relu(self.fc1(x)))
        x = self.conv2_drop(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x
        

In [18]:
model = Net()
optimizer = optim.RMSprop(model.parameters(),lr=0.00001)
optimizer = optim.Adam(model.parameters(),lr=0.01)
MSELoss = torch.nn.MSELoss()

In [15]:
def train(epoch):
    model.train()
    for batch_idx, (batch_data, batch_target) in enumerate(train_loader):
        batch_data, batch_target = Variable(batch_data.float()), Variable(batch_target.float())
        optimizer.zero_grad()
        output = model(batch_data)
        #loss = F.nll_loss(output, batch_target)
        loss = MSELoss(output, batch_target)
        loss.backward()
        optimizer.step()
        #if batch_idx % LOG_INTERVAL == 0:
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(batch_data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def test():
    model.eval()
    test_loss = 0
    ammount = 0
    for data, target in val_loader:
        data, target = Variable(data.float(), volatile=True), Variable(target.float())
        output = model(data)
        ammount += len(data)
        test_loss += MSELoss(output, target).data[0]
        
    test_loss /= ammount
    print('Test set: Average loss: %s\n' % test_loss)
    

In [19]:
for epoch in range(1, EPOCHS + 1):
    train(epoch)
    test()

Train Epoch: 1 [5460/5639 (99%)]	Loss: 46.306877
Test set: Average loss: 0.28740975738417174

Train Epoch: 2 [5460/5639 (99%)]	Loss: 51.194416
Test set: Average loss: 0.2933779425654851

